<a href="https://colab.research.google.com/github/kristianbagus/project/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation System

Sistem rekomendasi merupakan hal yang cukup umum digunakan pada sebuah website. Sistem Rekomendasi Sederhana, seperti namanya adalah sistem rekomendasi yang hanya menggunakan urutan sebagai dasar perhitungannya.Sebagai contoh, rekomendasi '5 film terbaik' menggunakan urutan berdasarkan mungkin vote terbanyak, rating tertinggi, penjualan film paling tinggi, atau apapun yang lain. Dalam kasus ini, kita akan menggunakan kombinasi antara rata-rata rating, jumlah vote, dan membentuk metric baru dari metric yang sudah ada, kemudian kita akan melakukan sorting untuk metric ini dari yang tertinggi ke terendah.

Di IMDB sendiri, mereka menggunakan sistem Weighted Average untuk menghitung rating dari sebuah film. Ide awal di balik sistem rekomendasi ini adalah sebagai berikut.

1. Film-film yang lebih populer akan memiliki kemungkinan yang lebih besar untuk disukai juga oleh rata-rata penonton.
2. Model ini tidak memberikan rekomendasi yang personal untuk setiap tipe user. 
3. Implementasi model ini pun juga bisa dibilang cukup mudah, yang perlu kita lakukan hanyalah mengurutkan film-film tersebut berdasarkan rating dan popularitas dan menunjukkan film teratas dari list film tersebut.

Formula dari IMDB dengan Weighted Rating:

  $Weighted  Rating = \frac{v}{(v+m)}  R + \frac{m}{(v+m)} C$

dimana:

    v: jumlah votes untuk film tersebut
    m: jumlah minimum votes yang dibutuhkan supaya dapat masuk dalam chart
    R: rata-rata rating dari film tersebut
    C: rata-rata jumlah votes dari seluruh film yang ada


## Importing Data

In [1]:
#import library yang dibutuhkan
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

#lakukan pembacaan dataset
movie_df = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/title.basics.tsv', sep='\t') #untuk menyimpan title_basics.tsv
rating_df = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/title.ratings.tsv', sep='\t') #untuk menyimpan title.ratings.tsv

### Cek Dataframe Movie

In [2]:
movie_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0221078,short,"Circle Dance, Ute Indians","Circle Dance, Ute Indians",0,1898,\N,\N,"Documentary,Short"
1,tt8862466,tvEpisode,"¡El #TeamOsos va con todo al ""Reality del amor""!","¡El #TeamOsos va con todo al ""Reality del amor""!",0,2018,\N,\N,"Comedy,Drama"
2,tt7157720,tvEpisode,Episode #3.41,Episode #3.41,0,2016,\N,29,"Comedy,Game-Show"
3,tt2974998,tvEpisode,Episode dated 16 May 1987,Episode dated 16 May 1987,0,1987,\N,\N,News
4,tt2903620,tvEpisode,Frances Bavier: Aunt Bee Retires,Frances Bavier: Aunt Bee Retires,0,1973,\N,\N,Documentary


In [3]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          9025 non-null   object
 1   titleType       9025 non-null   object
 2   primaryTitle    9011 non-null   object
 3   originalTitle   9011 non-null   object
 4   isAdult         9025 non-null   int64 
 5   startYear       9025 non-null   object
 6   endYear         9025 non-null   object
 7   runtimeMinutes  9025 non-null   object
 8   genres          9014 non-null   object
dtypes: int64(1), object(8)
memory usage: 634.7+ KB


### Cek Dataframe Rating

In [4]:
rating_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1608
1,tt0000002,6.0,197
2,tt0000003,6.5,1285
3,tt0000004,6.1,121
4,tt0000005,6.1,2050


In [5]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030009 entries, 0 to 1030008
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1030009 non-null  object 
 1   averageRating  1030009 non-null  float64
 2   numVotes       1030009 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 23.6+ MB


## Data Preprocessing

### Missing Value

Jika kita perhatikan pada dataframe movie, terdapat beberapa data dengan nilai '\N'. Nilai '\N' di sini berarti NULL. Oleh karena itu, kita perlu mengubahnya menjadi nilai NaN supaya dapat dideteksi sebagai missing value.



In [6]:
movie_df = movie_df.replace('\\N', np.nan)


Setelah mengubah nilai '\N' menjadi NaN, hal selanjutnya yang akan kita lakukan adalah melihat berapa banyak data bernilai NULL pada masing-masing kolom yang ada pada table movie (movie_df)


In [7]:
print(movie_df.isnull().sum())

tconst               0
titleType            0
primaryTitle        14
originalTitle       14
isAdult              0
startYear          673
endYear           8946
runtimeMinutes    6424
genres             742
dtype: int64


Karena nantinya kita memerlukan kolom seperti 'primaryTitle', 'originalTitle', 'startYear', 'genres' untuk melakukan rekomendasi, maka kita bisa menghapus baris yang memiliki nilai NaN pada kolom-kolom tersebut.

In [8]:
movie_df = movie_df.dropna(subset=['primaryTitle', 'originalTitle', 'startYear', 'genres'])
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7685 entries, 0 to 8999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          7685 non-null   object
 1   titleType       7685 non-null   object
 2   primaryTitle    7685 non-null   object
 3   originalTitle   7685 non-null   object
 4   isAdult         7685 non-null   int64 
 5   startYear       7685 non-null   object
 6   endYear         74 non-null     object
 7   runtimeMinutes  2487 non-null   object
 8   genres          7685 non-null   object
dtypes: int64(1), object(8)
memory usage: 600.4+ KB


### Change Datatype

Jika dilihat pada info tabel movie_df, terdapat beberapa kolom yang tipe datanya tidak sesuai seperti kolom startYear, endYear, dan runtimeMinutes. Oleh karena itu, kolom-kolom ini akan diganti tipe datanya seperti yang seharusnya (float64).

In [9]:
movie_df = movie_df.astype({'startYear': 'float64', 'endYear': 'float64', 'runtimeMinutes': 'float64'})
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7685 entries, 0 to 8999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          7685 non-null   object 
 1   titleType       7685 non-null   object 
 2   primaryTitle    7685 non-null   object 
 3   originalTitle   7685 non-null   object 
 4   isAdult         7685 non-null   int64  
 5   startYear       7685 non-null   float64
 6   endYear         74 non-null     float64
 7   runtimeMinutes  2487 non-null   float64
 8   genres          7685 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 600.4+ KB



### Mengubah Genres Menjadi List

Untuk membuat rekomendasi berdasarkan genre, selanjutnya kita akan membuat sebuah function yang bernama transform_to_list untuk mengubah nilai genre menjadi list. 


In [10]:
def transform_to_list(x):
    if ',' in x: 
    #ubah menjadi list apabila ada data pada kolom genre
        return x.split(',')
    else: 
    #jika tidak ada data, ubah menjadi list kosong
        return []

movie_df['genres'] = movie_df['genres'].apply(lambda x: transform_to_list(x))


### Join Table Movie dan Table Rating

Setelah selesai memproses tabel movie, kita baru bisa melakukan join antara rating_df dan movie_df untuk mendapatkan rating pada setiap film yang tersedia.


In [11]:
#Lakukan join pada kedua table
movie_rating_df = pd.merge(movie_df, rating_df, on='tconst', how='inner')

#Tampilkan 5 data teratas
movie_rating_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0043745,short,Lion Down,Lion Down,0,1951.0,NaN,7.0,"[Animation, Comedy, Family]",7.1,459
1,tt0167491,video,Wicked Covergirls,Wicked Covergirls,1,1998.0,NaN,85.0,[],5.7,7
2,tt6574096,tvEpisode,Shadow Play - Part 2,Shadow Play - Part 2,0,2017.0,NaN,22.0,"[Adventure, Animation, Comedy]",8.5,240
3,tt6941700,tvEpisode,RuPaul Roast,RuPaul Roast,0,2017.0,NaN,NaN,[],8.0,11
4,tt7305674,video,UCLA Track & Field Promo,UCLA Track & Field Promo,0,2017.0,NaN,NaN,"[Short, Sport]",9.7,7


In [12]:
#Tampilkan tipe data dari tiap kolom
movie_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1345 entries, 0 to 1344
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          1345 non-null   object 
 1   titleType       1345 non-null   object 
 2   primaryTitle    1345 non-null   object 
 3   originalTitle   1345 non-null   object 
 4   isAdult         1345 non-null   int64  
 5   startYear       1345 non-null   float64
 6   endYear         25 non-null     float64
 7   runtimeMinutes  992 non-null    float64
 8   genres          1345 non-null   object 
 9   averageRating   1345 non-null   float64
 10  numVotes        1345 non-null   int64  
dtypes: float64(4), int64(2), object(5)
memory usage: 126.1+ KB


## Recommender System

Setelah melakukan processing data, selanjutnya kita bisa membuat sistem rekomendasi yang diinginkan.


### Nilai C

Hal pertama yang akan dilakukan untuk membuat sistem rekomendasi adalah mencari nilai dari C yang merupakan rata-rata dari averageRating.


In [13]:
C = movie_rating_df['averageRating'].mean()
print(C)

6.8731598513011205


### Nilai m

Setelah mencari nilai C, kita akan mencari nilai m

Pada contoh kali ini, mari kita ambil film dengan numVotes di atas 70% populasi, jadi populasi yang akan kita ambil hanya sebesar 30%. 


In [14]:
m = movie_rating_df['numVotes'].quantile(0.7)
print(m)

64.0


### Membuat Fungsi Weighted Formula

Selanjutnya kita dapat membuat sebuah fungsi untuk menghitung weighted rating.


In [15]:
def imdb_weighted_rating(df, var=0.7):
    v = df['numVotes']
    R = df['averageRating']
    C = df['averageRating'].mean()
    m = df['numVotes'].quantile(var)
    df['score'] = (v/(m+v))* R + (m/(m+v))* C #Rumus IMDb 
    return df['score']
    
imdb_weighted_rating(movie_rating_df)

#melakukan pengecekan dataframe
movie_rating_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,score
0,tt0043745,short,Lion Down,Lion Down,0,1951.0,NaN,7.0,"[Animation, Comedy, Family]",7.1,459,7.072241
1,tt0167491,video,Wicked Covergirls,Wicked Covergirls,1,1998.0,NaN,85.0,[],5.7,7,6.757496
2,tt6574096,tvEpisode,Shadow Play - Part 2,Shadow Play - Part 2,0,2017.0,NaN,22.0,"[Adventure, Animation, Comedy]",8.5,240,8.157507
3,tt6941700,tvEpisode,RuPaul Roast,RuPaul Roast,0,2017.0,NaN,NaN,[],8.0,11,7.038430
4,tt7305674,video,UCLA Track & Field Promo,UCLA Track & Field Promo,0,2017.0,NaN,NaN,"[Short, Sport]",9.7,7,7.151862



### Membuat Simple Recommender System

Dari fungsi yang sudah kita buat sebelumnya, kita mendapat kolom baru bernama 'score'.

Untuk membuat sistem rekomendasi yang sederhana, kita bisa mengurutkannya melalui rating dan memfilter dengan batas jumlah votes yang telah ditentukan sebelumnya.

In [16]:
def simple_recommender(df, top=100):
    df = df.loc[df['numVotes'] >= m]
    df = df.sort_values(by='score', ascending=False) #urutkan dari nilai tertinggi ke terendah
    
    #Ambil data 100 teratas
    df = df[:top]
    return df


### Membuat Simple Recommender System dengan User Preferences

Setelah membuat sistem rekomendasi yang sederhana, kita bisa membuat lagi sistem rekomendasi menurut preferensi pengguna dengan menambahkan filter lain seperti ask_adult, ask_start_year, ask_genre.


In [17]:
def user_prefer_recommender(df_r, ask_adult, ask_start_year, ask_genre, top=100):

    df = df_r.copy()

    #ask_adult = yes/no
    if ask_adult.lower() == 'yes':
        df = df.loc[df['isAdult'] == 1]
    elif ask_adult.lower() == 'no':
        df = df.loc[df['isAdult'] == 0]

    #ask_start_year = numeric
    df = df.loc[df['startYear'] >= int(ask_start_year)]

    #ask_genre = 'all' atau yang lain
    if ask_genre.lower() == 'all':
        df = df
    else:
        def filter_genre(x):
            if ask_genre.lower() in str(x).lower():
                return True
            else:
                return False
        df = df.loc[df['genres'].apply(lambda x: filter_genre(x))]

    
    #Mengambil film dengan numVotes yang lebih besar atau sama dengan nilai m 
    df = df.loc[df['numVotes'] >= m]  
    df = df.sort_values(by='score', ascending=False)
    
    #jumlah data yang diambil
    df = df[:top]
    return df

## Mengaplikasikan Recommender System 

Setelah selesai untuk membuat sistem rekomendasi, kita bisa menggunakannya pada case tertentu.

### Simple Recommender System

Ada seseorang yang baru memasuki website imdb, user tersebut ingin melihat 20 judul dengan rating terbaik yang ada di imdb. Dengan begitu rekomendasi untuk user tersebut adalah:

In [18]:
simple_recommender(movie_rating_df, top=10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,score
66,tt4110822,tvEpisode,S.O.S. Part 2,S.O.S. Part 2,0,2015.0,NaN,43.0,"[Action, Adventure, Drama]",9.4,3820,9.358363
1122,tt5807780,videoGame,Spider-Man,Spider-Man,0,2018.0,NaN,NaN,"[Action, Adventure, Fantasy]",9.3,10104,9.284725
226,tt2200252,video,Attack of the Clones Review,Attack of the Clones Review,0,2010.0,NaN,86.0,[],9.3,1411,9.194700
1152,tt7697962,tvEpisode,Chapter Seventeen: The Missionaries,Chapter Seventeen: The Missionaries,0,2019.0,NaN,54.0,"[Drama, Fantasy, Horror]",9.2,1536,9.106926
314,tt7124590,tvEpisode,Chapter Thirty-Four: Judgment Night,Chapter Thirty-Four: Judgment Night,0,2018.0,NaN,42.0,"[Crime, Drama, Mystery]",9.1,1859,9.025888
69,tt8399426,tvEpisode,Savages,Savages,0,2018.0,NaN,58.0,"[Drama, Fantasy, Romance]",9.0,1428,8.908768
1018,tt0533506,tvEpisode,The Prom,The Prom,0,1999.0,NaN,60.0,"[Action, Drama, Fantasy]",8.9,2740,8.853738
1205,tt2843830,tvEpisode,VIII.,VIII.,0,2014.0,NaN,57.0,"[Adventure, Drama]",8.9,1753,8.828609
1338,tt11143130,tvEpisode,Agar Tum Saath Ho,Agar Tum Saath Ho,0,2019.0,NaN,45.0,"[Comedy, Drama, Romance]",9.6,132,8.709603
1060,tt4295140,tvSeries,Chef's Table,Chef's Table,0,2015.0,NaN,50.0,[],8.6,12056,8.590881


### Simple Recommender System dengan User Preferences

User tersebut ingin mencari lagi secara spesifik tentang film sesuai dengan kesukaannya. Dia menginginkan film yang tidak memiliki unsur dewasa, bergenre drama, dan rilis setelah tahun 2000. Dengan begitu rekomendasi untuk user tersebut adalah:

In [19]:
user_prefer_recommender(movie_rating_df, 
                        ask_adult = 'no',
                        ask_start_year = 2000,
                        ask_genre = 'adventure',
                        top = 10
                       )

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,score
66,tt4110822,tvEpisode,S.O.S. Part 2,S.O.S. Part 2,0,2015.0,NaN,43.0,"[Action, Adventure, Drama]",9.4,3820,9.358363
1122,tt5807780,videoGame,Spider-Man,Spider-Man,0,2018.0,NaN,NaN,"[Action, Adventure, Fantasy]",9.3,10104,9.284725
1205,tt2843830,tvEpisode,VIII.,VIII.,0,2014.0,NaN,57.0,"[Adventure, Drama]",8.9,1753,8.828609
345,tt4084774,tvEpisode,Trial and Punishment,Trial and Punishment,0,2015.0,NaN,56.0,"[Adventure, Drama]",8.8,289,8.450658
947,tt0340374,videoGame,Splinter Cell,Splinter Cell,0,2002.0,NaN,NaN,"[Action, Adventure, Thriller]",8.3,2791,8.268015
1188,tt3642464,tvEpisode,Giant Woman,Giant Woman,0,2014.0,NaN,11.0,"[Adventure, Animation, Comedy]",8.4,566,8.244892
1314,tt6644294,tvEpisode,The Hostile Hospital: Part Two,The Hostile Hospital: Part Two,0,2018.0,NaN,40.0,"[Adventure, Comedy, Drama]",8.3,812,8.195756
2,tt6574096,tvEpisode,Shadow Play - Part 2,Shadow Play - Part 2,0,2017.0,NaN,22.0,"[Adventure, Animation, Comedy]",8.5,240,8.157507
767,tt4279086,tvEpisode,And Santa's Midnight Run,And Santa's Midnight Run,0,2014.0,NaN,42.0,"[Action, Adventure, Comedy]",8.2,823,8.104264
1084,tt4174072,tvEpisode,Immortal Emerges from Cave,Immortal Emerges from Cave,0,2017.0,NaN,53.0,"[Action, Adventure, Crime]",8.0,2898,7.975652
